In [39]:
from CallProcessor import CallProcessor
import pandas as pd
from utils import extract_phrases,jsonify
from Mongo_DB import Mongo_DB
from collections import Counter
import requests
import json
import os
import re
import string
from collections import defaultdict

In [45]:
class Interface:
    def __init__(self,keyword_file='insurance.json'):
        df_dict = pd.read_json(keyword_file).to_dict()
        keywords = extract_phrases(df_dict)
        self.call_processor = CallProcessor(keywords=keywords)
        # self.DB = Mongo_DB() # paramaeters are inserted in Construct
        self.DB = Mongo_DB(address='mongodb://localhost:27017/',
                 db_name='call_analytics_tool',
                 collection_name='call_record5',)
        
    def get_most_common(self):
        data = self.DB.find({},['spliited_trans'])
     
        # Initialize an empty list to store all cleaned phrases
        all_phrases = []

        # Loop through each entry in the data list to collect all phrases
        for entry in data:
            for key in entry['spliited_trans']['splitted_transcript']:
                phrases = entry['spliited_trans']['splitted_transcript'][key]
                for phrase in phrases:
                    all_phrases.append(phrase)

        # Clean up each phrase by removing special characters and Filter out phrases with less than 4 words
        cleaned_phrases = []
        for phrase in all_phrases:
            if len(phrase.split()) >= 4:
                clean_phrase = re.sub(r'[^a-zA-Z0-9\s]', '', phrase)
                cleaned_phrases.append(clean_phrase)

        # Count occurrences of each phrase
        phrase_counts = Counter(cleaned_phrases)

        # Sort phrases by their counts
        sorted_phrases = sorted(phrase_counts.items(), key=lambda x: x[1], reverse=True)

        # Extract only the top 5 phrases
        result = dict(sorted_phrases[:5])

        return result


interface = Interface()
interface.get_complete_data()

{' Hi this is Ella with American Senior Citizens Care How are you doing today': 3,
 ' Okay Okay This call is about a new state regulated final expense insurance plan which covers 100 of your burial funeral or cremation expenses It is specifically designed for the people on fixed income or social security Would you like to learn more about it Yes': 3,
 ' May I ask how old are you So Nicole Are you capable enough to make your own financial decisions Yes Great Let me bring the state licensed product specialists on the line to share information with you Okay Okay': 3,
 ' to face internally and SEC Northwest': 1,
 ' Hello my name is Ducky from New York How are you today': 1}